# Bayesian Statistics

### [1] Concept of framework:
- Bayesian inference is reallocation of probabilities across possibilities
- Possibilities are parameter values (a, u, o) in meaningful mathmatical models N(u, a)

### [2] Steps of Bayesian Data Analysis:

- (1) Indentify data -> measurement, scale, predictor, target
- (2) Define a mathmatical form / parameters to describe in appropriate theoretical purpose
- (3) Spcify a prior distribution on parameters
- (4) Use Bayesian inference to re-allocate probabilities across parameter values and generate postieror distribution
- (5) Check the posterior prediction under reasonable accuracy

### [3] What is Probability?
- Sample space – A set of outcome exhausted all possibilities and mutually exclusive
- Probability – is a way of assigning numbers to a set of mutually exclusive possibilities
- Probability Distribution – A set of all possibilities and their corresponding probabilities
-      (1) Discrete Outcomes (Probability Mass)
-      (2) Continuous Outcomes (Probability Density)
- Probability Density Function – Any function that has only nonnegative values and integrates to 1can be constructed as a probability density function
- Expected Value of a Distribution – Weight each outcome value with Prod()  them sum up, usually “mean”


### [4] Bayes' Rule
- P ( Theta | Data )   =   P ( Data | Theta )    *   P ( Theta )   /   P ( Data )
- Posterior Probability   =   Likelihood   *   Prior    /   Evidence ( Marginal Likelihood ) 


### [5] How to do Bayesian Inference?

- (1) Conjugate Prior – Mathematical update from prior to posterior, prior distribution the same as posterior distribution

- (2) Grid Approximation – Not all prior can be expressed in a conjugate prior, we can create a grid to discrete values over Theta, Create value of this binned “Mass function”. Can still be difficult if there are multiple Thetas

- (3) Markov Chain Monte Carlo (MCMC) – {MC} – each step is independent, {MC} – Assessing properties of a target distribution by generating representative random values. Approximating a posterior distribution by collecting a large representative sample from it. 

##### Assumption: 
prior distribution specified by function; likelihood specified by function; Both can be easily computed up to a multiplicative constant 
##### Result: 
an approximation of posterior distribution P ( Theta | data )

##### Algorithms: 

###### Metropolis: 

Step1 – Pick an arbitrary initial Theta or {Theta[ij]}

Step2 – Based on an proposal distribution – propose a move, tune sd, fewer 
steps to converge 

Step3 – If P(D|theta-new)*P(theta-new) > P(D|theta-old)*P(theta-old) then move theta to theta-new; else then randomly select # from [0-1], if # between [0, P(D|theta-new)*P(theta-new) / P(D|theta-old)*P(theta-old)], then move theta to theta-new, else then stay theta-old.

##### Gibbs Sampling:

Good for multiple parameters, hierarchy model \ limit (high correlated parameters)

Step1 – Pick a an arbitrary initial value point of combination theta[ij] 

Step2 – from theta[ij] circle through each of them, select theta[ij=1] then generate random value from conditional probability distribution P( theta[ij=1] | theta[ij<>1], D ) ** proposal distribution mimic posterior
Step3 – use theta-new[ij=1] + theta-old[ij<>1] as proposed theta move
Step4 – same as Metropolis

###### Hamiltonian Monte Carlo (HMC):

Good for large complex models
Variation of “Metropolis” – Different proposal distribution that changes depends on the current position. HMC figures out the direction in which the posterior distribution increases, called its gradient, and warps the proposal distribution towards the gradient.

##### Evaluate / Optimization on MCMC Process
- Process:

Step1 – Initiate Chains

Step2 – Generate Chains

Step3 – Examine the Chains

Step4 – Shows MCMC sample from the posterior distribution

- Measurement:

Trace Plot – Good overlap of chains – burn-in

Density Plot – Plot dist of each chains, 95HDI overlap

Autocorrelation Plot – “How clumpy?”, very close to each

Shrink Factor Plot - ??, close to each

### [6] How to evaluate performance - Goal, Power, Sample Size?

###### Statistical Power – 

Is the probability of achieving the goal of a planned empirical study, if a suspected underlying state of the world is true, (1) unachievable – Don’t waste time. (2) Easily achievable – Don’t sample too much. ** Traditional NHST in multiple tests, Power decreases while in Bayesian, power stable because it totally based on data 

##### Goals and Obstacles – 

Goals: (1) Reject a null value of a parameter; (2) Affirm a predicted value of a parameter ([ROPE] included in HDI); (3) Achieve precision in the estimation of parameter (HDI width less than a threshold) 

Obstacles: random sample is only a probabilistic representation of the real world. Noises are the enemies.

##### Power – 

Since the random noises, the goal can only be achieve probabilistically. The probability of achieving the goal, given the state-of-the world and sampling plan, is called “Power” of the planned research  **Traditional NHST – power’s only goal rejecting null 

- Ways to increase the power: (1) Control on co-founder to detect noisy; (2) Increase underlying effect – increase test effect; (3) Increase sample size

##### Power Analysis – 

Hypothesis Parameter distribution (Good if summarized from previous data) Sample a Parameter value Use that Parameter value to simulate the sampling data (Like the planned sampling process)  Prior ~ Likelihood ~ Posterior  Evaluate of all iterations ?% achieved the goal = ex. 70% the power  **Only useful when simulated data close to real data

###### Sample Size – 
More sample size, more power | It increase the precision NOT the value, if False, more data can’t change.

##### Sampling Bias: 
No Bias (Sample N cases); Biased (Sample Z success events); No Bias (Sample cases for M hours)

##### Other Expressions of Goals – 
Ways to express mathematically the goal of precision in estimation (Not using power)

- Average Length Criterion – Average width of the 95HDI in posterior

- Average Coverage Criterion – % of Iteration Given specific width of HDI, require its Mass exceeds 95%

- Entropy of a distribution – Average entropy (The more spread a distribution the bigger the entropy) – Higher precision, low entropy

##### Other Expressions of Goals – 
Ways to express mathematically the goal of excluding a Null value

- Sufficient large Bayes Factor (BF) – Average of BF from posterior

- How often % HDI excludes ROPE - % confidence to reject null


##### Sequential Testing
Keep sampling while using these metrics as “Stopping Rule” (P-value, Criterial BF, HDI-ROPE, Precision)

- P-Value: Always eventually reject null even if null is true (100% fa)
- Criterial BF: No 100% fa when null is true, yet, also falsely accept when null is not true
- HDI_ROPE: No 100% fa when null is true, No falsely accept
- Precision (HDI_Width): Never fa, but may be uncertain in some case



## JAGS - Software

xxxxxxxxxx

In [ ]:
# --------- R

>>>>>>>>> JAGS 
install.packages("rjags")
install.packages("runjags")
library(rjags);library(runjags)

# Load data
myData = read.csv("xxx.csv")
y = myData$y
Ntotal = length(y)
dataList = list(
	y = y,
	Ntotal = Ntotal
	)

# Specify the model
modelString = """ # open quote for modelString
   model {
   	for ( i in 1:Ntotal ) {
   		y[i] ~ dbern( theta ) 
   	}
   	theta ~ dbeta( 1 , 1 )
   }

"""
writeLines(modelString, con="TEMPmodel.txt") # write to file

# Initialize chains
initsList = function() {
	resampledY = sample(y, replace=T)               # resample values from y
	thetaInit = sum(resampledY)/length(resampledY)  # compute proportion (MLE)
	thetaInit = 0.001+0.998*thetaInit               # keep away from 0, 1
	return(list(theta=thetaInit))                   # return as a named list
}

# Generate chains
jagsModel = jags.model(file="TEMPmodel.txt", 
	                   data=dataList, 
	                   inits=initsList,
	                   n.chains=3,
	                   n.adapt=500)

update(jagsModel, n.iter=500) # run chains for number of steps

# Tell JAGs to generate MCMC sample we used to represent the posterior distribution
codaSamples = coda.samples(jagsModel, variable.names=c("theta"), n.iter=3334) 

# Exame Chains


In [ ]:
# --------- Python

## STAN - Software

xxxxxxxxx

In [ ]:
# -------- R

In [ ]:
# -------- Python

## ------------------------------------ [Hierarchical Models]

###### Hierarchical Model - 
HM are mathematical descriptions involving multiple parameters such that the credible values of some parameters meaningfully depend on the values of other parameters

###### Why very effective? – 
estimate of each individual parameter is simultaneously informed by data from all other individuals. All individual inform the higher level parameter – constrain all individual parameters 

##### Parameters at different levels in a hierarchical model 
are all merely parameters that coexists in a joint parameter space, we simply apply Bayes’ rule to the joint parameters space ----- P ( Theta1, Theta2, beta1, beta2 | Data ) = P ( Data | Theta1) * P ( Theta1 | Theta2 ) * P ( theta2 | beta1 ) * P ( beat1 | beta2) * P ( beta2 )

### Example 1 – Single Coin from a Single Mint

Y[i]: [1,1,0,1,0, …, 1]

For every Y in [i] generated from distribution “dbern(Theta)”  

“Theta” generated from distribution “Beta(w,k)”

“w” generated from “Beta(Aw, Bw)”

“k” generated from “Gamma(Sk,Rk)”

Aw, Bw, Sk, Rk are constant


### Example 2 – Multiple Coins(s) from a Single Mint

Y1[i]: [1,1,0,1,0, …, 1] … Ys[i]: [1,1,0,1,0, …, 1] 

For every Y in [i|s] generated from distribution “dbern(Theta[s])”  

For every “Theta” in [s] generated from distribution “Beta(w,k)”

“w” generated from “Beta(Aw, Bw)”

“k” generated from “Gamma(Sk,Rk)”

Aw, Bw, Sk, Rk are constant

### Example 3 – Multiple Coins(s) from a Multiple Mints(c)

Y1|1[i]: [1,1,0,1,0, …, 1] … Ys|c[i]: [1,1,0,1,0, …, 1]

For every Y in [i|s,c] generated from distribution “dbern(Theta[s|c])”  

For every “Theta” in [s|c] generated from distribution “Beta(w[c],k[c])”

For every “w” in [c] generated from “Beta(Aw, Bw)”

For every  “k” in [c[ generated from “Gamma(Sk,Rk)”

Aw, Bw, Sk, Rk are constant

### Example 4 – Model Comparison 

Re-allocate credibility across models – Top-level parameter model index [m] evenly distributed

A model = likelihood function  *  Prior Distribution  

Define by Bayes Factor (BF) – P(D|M=1) / P(D|M=2) **Probability of data in M1, M2

Sensitive to prior choice:

- Choice 1: Parameters for all Ms in joint space
- Choice 2: Likelihood, prior sep(M for M)
- Choice 3: Prior sep(M for M) but likehood the same

###### Evaluation1 – Conjugate BF: 

Posterior Odd – P (M=1|D) / P (M=2|D)

Bayes Factor – P (D|M=1) / P (D|M=2) 

Prior Odd – P (M=1) / P (M=2)

Posterior Odd = BF * Prior Odd

Conjugate BF – P(z,n) = P(z+a,n-z+b) = P (D|M)

Calculate Posterior Odd = 0.213

Since SUM(p(m[i]|D)) = 1, so P(M=1|D)/(1-P(M=1|D)) = 0.213, P(M=1|D) = 17.6%, P(M=2|D) = 1 – 17.6% = 82.4%

##### Evaluation2 - 
MCMC: prior M = 0.5/0.5, then posterior distribution of index [m], ?/? which more likely?

## ------------------------------------ [Hull Hypothesis Significance Testing]

##### NHST (Frequentist): 
P-value, is the probability of getting an sample outcome from the hypothesis population (t/z-distribution) that is as extreme as or more extreme than the actual outcome (data) when using the intended sampling and testing process

##### NHST (Bayesian): 
Doesn’t depend on intension of testing / collecting. Assume data independent across each other. So, the probability of the set of the data is simply the product of probability of the individual outcome. [likelihood] * [prior] = [posterior] -> check HDI contains null | Check “BF”

##### Example 1 – One sample Tests

- Estimation Approach

[Likelihood] * [Prior]  MCMC  [Posterior]

If [HDI] not contain [ROPE]-null  Reject

If [HDI] contain [ROPE]-null  Accept

If [HDI] contain by [ROPE]-null  Accept (weak)

If [HDI] overlap with [ROPE]-null  Not Clear


- Model-Comparison Approach

M1: [Likelihood D]  *  [Prior - null]

M2: [Likelihood D]  *  [Prior – alter]

Posterior Odd  = BF * Prior Odd

Posterior Odd = 0.342

Since SUM(p(m[i]|D)) = 1, so P(M1|D)/(1-P(M1|D)) = 0.342

P(M=1|D) = 27.6%

P(M2|D) = 1 – 27.6% = 72.4%


#### Example 2 – Two or more sample[s] Tests

- Estimation Approach

Hierarchical Model - like [Multiple coins[s] from single mint]

[Posterior]: Theta1_Theta2_Diff -> [HDI] vs {ROPE-null:0] ?


- Model-Comparison Approach

M1: each group has distinct Theta[s]

M2: all groups share one Theta0

MCMC -- [Posterior]-M distribution -> Which M more creditable?

BF (M1, M2) ?

## ------------------------------------ [The Generalized Linear Models]

This family of models comprises the traditional “off the shell” analyses such as T Test, ANOVA, Multiple Linear Regression, Logistic, Log-linear Models, etc.

- X,Y (Type) – Metric(1-infinite), Ordinal(1,2,3), Nominal(A,B,C), Count(12,45,31) 

- Metric = Y~Normal(u,a)\T(v,u,a)

- Dichotomous = Y~Bernoulli(u)

- Nominal = Y~MASS_DIS(…u[j]….)

- Ordinal = Y~MASS_DIS(…u[j]….)

- Count = Y~Poisson(u)

#### [1] Metric Predicted Variable on One or Two Groups

T-Distribution -> metric (Robust than Normal-Distribution)

Z-Distribution -> Ratio

###### One Group: 

Y[i] distributed by T-distribution(v,u,a)

v distributed by Exp(K)

u distributed by N(M,S)

a distributed by Uni(L,H)

v,u,a are Constants

Evaluation: MCMC Posterior(u) HDI vs Null value



###### Two Groups: 

Y[i]|j distributed by T-distribution(v[j],u[j],a[j])

For v in jth distributed by Exp(K)

For u in jth distributed by N(M,S)

For a in jth distributed by Uni(L,H)

v,u,a are Constants

Evaluation: MCMC Posterior(u[1]_u[2]_Diff) HDI vs 0

#### [2] Metric Predicted Variable with One Metric Predictor

Simple Linear Regression 1X

Y[i] distributed by T-distribution(v,u[i],a)

u[i] calculated from ( beta0 + beta1*x[i] )

beta0 distributed by N(M0,S0)

beta1 distributed by N(M1,S1)

v distributed by Exp(K)

a distributed by Uni(L,H)

M0, S0, M1, S1, u, a are Constants

Evaluation: MCMC – Posterior(u[i]) of ith Y

#### [3] Metric Predicted Variable with Multiple Metric Predictors

Multiple Linear Regression [j] Xs

Y[i] distributed by T-distribution(v,u[i],a)

u[i] calculated from ( beta0 + beta[j]*x[i|j] )

beta0 distributed by N(M0,S0)

For beta in jth distributed by N(M[j],S[j])

v distributed by Exp(K)

a distributed by Uni(L,H) **Assume variance the same

M0, S0, M[j], S[j], u, a are Constants

Evaluation: MCMC – Posterior(u[i]) of ith Y

#### [4] Metric Predicted Variable with One Nominal Predictor

- One-Way ANOVA: beta0 + beta1[j] X(nominal)

Y[i] distributed by N(u[i],ay)

u[i] calculated from beta0 + beta1[j] X[i](nominal)

beta0 distributed by N(M0, S0)

For beat1 in jth distributed by N(0, a-beta1)

ay distributed by Uni(L, H)

L, H, M0, S0 are constants

a-beta1 is constant / small-strong shrinkage

Evaluation: MCMC – Posterior(u[A]_u[B]_Diff) HDI vs 0

- ANCOVA: beta0 + beta1[j] X1(nominal) + beta2 X2(covariance)

#### [5] Metric Predicted Variable with Multiple Nominal Predictors

MANOVA: beta0 + beta1[j] X1(nominal) + beta2[k] X2(nominal) + beta3[j*k] X1*2[j*k]

Y[i] distributed by N(u[i], ay)

u[i] calculated from “beta0 + beta1[j] X1[i](nominal) + beta2[k] X2[i](nominal) + beta3[j*k] X1*2[i][j*k]”

beta0 distributed by N(M0, S0)

For beta1 in jth distributed by N(0, a-beta1)

For beta2 in kth distributed by N(0, a-beta2)

For beta3 in j*kth distributed by N(0, a-beta3)

ay distributed by Uni(L, H)

L, H, M0, S0 are constants

a-beta1, a-beta2, a-beta3 are constant / small-strong shrinkage

Evaluation: MCMC – Posterior(u[A]_u[B]_Diff) HDI vs 0  Main effect

Evaluation: MCMC – Posterior(u[A1]_u[A2]_vs_u[B]) HDI vs 0  Interaction effect

#### [6] Dichotomous Predicted Variable (As logistic regression)

- Logistic Regression: Logistic( beta0 + beta[j] X[j](metric)) 

Y[i] distributed by dbern(u[i])

u[i] calculated from “Logistic( beta0 + beta[j] X[j]) ” *Log(link func)

beta0 distributed by N(M0, S0) 

For beta in jth distrusted by N(Mi, Si)

M0, S0, Mi, Si are constants

Evaluation: MCMC – Posterior(u[i]) – prob() – Prediction


- Logistic Regression: Logistic( beta0 + beta1[j] X1[j](nominal))

Y[i|j] distributed by Binominal(u[i|j]) ** computation efficient

u[i|j] distributed by Beta(w[j], k)

k distributed by Gamma(Sk, Rk)

For w[j] in jth calculated from “Logistic( beta0 + beta[j] X[j]) ” *Log(link func)

beta0 distributed by N(M0, S0) 

For beta in jth distrusted by N(0, a-beta)

M0, S0, Sk, Rk are constants

a-beta is constant / small-strong shrinkage

Evaluation: MCMC – Posterior(u[i]) – prob() - Prediction

#### [7] Nominal Predicted Variable

- Multinomial Logistic Regression (Soft-Max)



- Conditional Logistic Regression

#### [8] Ordinal Predicted Variable

#### [9] Count Predicted Variable